# 연습문제 4번

## 10-fold 교차검증법 ,  Leave-One-Out Cross Validation ( LOOCV )

### load Iris data

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

data_dir = Path('./data/bezdekIris.txt')
df = pd.read_table(data_dir, sep=',', header=None, names=['x0', 'x1', 'x2', 'x3', 'label'])

In [2]:
df

,x0,x1,x2,x3,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
df['label'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [4]:
y1 = df[df['label'] == 'Iris-setosa'].drop(['label'], axis=1)
y2 = df[df['label'] == 'Iris-versicolor'].drop(['label'], axis=1)
y3 = df[df['label'] == 'Iris-virginica'].drop(['label'], axis=1)

In [5]:
def concat_data(y1, y2):
    y1['y'] = 0
    y2['y'] = 1
    return pd.concat([y1, y2])

In [6]:
y1_y2 = concat_data(y1, y2)
y2_y3 = concat_data(y2, y3)
y1_y3 = concat_data(y1, y3)

In [7]:
y1_y3

,x0,x1,x2,x3,y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


### 10-fold 교차검증법

In [8]:
def k_fold_cross_validation(data, k=10):
    data = data.sample(frac=1).reset_index(drop=True)
    data_k = np.array_split(data, k)
    return data_k

### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression

y1_y2_k = k_fold_cross_validation(y1_y2)
y2_y3_k = k_fold_cross_validation(y2_y3)
y1_y3_k = k_fold_cross_validation(y1_y3)

# y1_y2
y1_y2_acc = []

for i in range(len(y1_y2_k)):
    test = y1_y2_k.pop(0)
    train = pd.concat(y1_y2_k)
    
    y1_y2_k.append(test)
    
    train_x = train.drop(['y'], axis=1)
    train_y = train['y']
    
    test_x = test.drop(['y'], axis=1)
    test_y = test['y']
    
    clf = LogisticRegression(random_state=0).fit(train_x, train_y)
    acc = clf.score(test_x, test_y)
    y1_y2_acc += [acc]
    
print(sum(y1_y2_acc) / len(y1_y2_acc))

1.0


In [10]:
# y2_y3
y2_y3_acc = []

for i in range(len(y2_y3_k)):
    test = y2_y3_k.pop(0)
    train = pd.concat(y2_y3_k)
    
    y2_y3_k.append(test)
    
    train_x = train.drop(['y'], axis=1)
    train_y = train['y']
    
    test_x = test.drop(['y'], axis=1)
    test_y = test['y']
    
    clf = LogisticRegression(random_state=0).fit(train_x, train_y)
    acc = clf.score(test_x, test_y)
    y2_y3_acc += [acc]
    
print(sum(y2_y3_acc) / len(y2_y3_acc))

0.95


### Leave-One-Out Cross Validation ( LOOCV )

In [13]:
# y1_y2
y1_y2_cor = 0

length =len(y1_y2)

for i in range(length):
    test = y1_y2.iloc[:1]
    train = y1_y2.iloc[1:]
    
    y1_y2 = pd.concat([train, test])
    
    train_x = train.drop(['y'], axis=1)
    train_y = train['y']
    
    test_x = test.drop(['y'], axis=1)
    test_y = test['y']
    
    clf = LogisticRegression(random_state=0).fit(train_x, train_y)
    pred = clf.predict(test_x)
    if pred[0] == test_y.values[0]:
        y1_y2_cor += 1

y1_y2_acc = y1_y2_cor / length

In [14]:
y1_y2_acc

1.0

In [15]:
# y2_y3
y2_y3_cor = 0

length =len(y2_y3)

for i in range(length):
    test = y2_y3.iloc[:1]
    train = y2_y3.iloc[1:]
    
    y2_y3 = pd.concat([train, test])
    
    train_x = train.drop(['y'], axis=1)
    train_y = train['y']
    
    test_x = test.drop(['y'], axis=1)
    test_y = test['y']
    
    clf = LogisticRegression(random_state=0).fit(train_x, train_y)
    pred = clf.predict(test_x)
    if pred[0] == test_y.values[0]:
        y2_y3_cor += 1

y2_y3_acc = y2_y3_cor / length

In [16]:
y2_y3_acc

0.96

In [36]:
y1_y2 = y1_y2.reset_index(drop=True)